In [20]:
import pandas as pd  
  
# assign data of lists.  
data = {'CountryName': ['United States', 'China', 'Germ', 'Brazil'], 'Age': [20, 21, 19, 18]}  
  
# Create DataFrame  
df = pd.DataFrame(data)  
  
# Print the output.  
df.head()

,CountryName,Age
0,United States,20
1,China,21
2,Germ,19
3,Brazil,18


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   CountryName  4 non-null      object
 1   Age          4 non-null      int64 
dtypes: int64(1), object(1)
memory usage: 192.0+ bytes


In [27]:
#function to convert to alpah2 country codes and continents
from pycountry_convert import country_alpha2_to_continent_code, country_name_to_country_alpha2
def get_continent(col):
    try:
        cn_a2_code =  country_name_to_country_alpha2(col)
    except:
        cn_a2_code = 'Unknown' 
    try:
        cn_continent = country_alpha2_to_continent_code(cn_a2_code)
    except:
        cn_continent = 'Unknown' 
    return (cn_a2_code, cn_continent)
df["codes"]=df["CountryName"].apply(get_continent)

In [28]:
df["Country"]=df["codes"].apply(lambda x:x[0])
df["Continent"]=df["codes"].apply(lambda x:x[1])
df.head()

,CountryName,Age,codes,Country,Continent
0,United States,20,"(US, NA)",US,NA
1,China,21,"(CN, AS)",CN,AS
2,Germ,19,"(Unknown, Unknown)",Unknown,Unknown
3,Brazil,18,"(BR, SA)",BR,SA


In [29]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="alpha")
def geolocate(country):
    try:
        # Geolocate the center of the country
        loc = geolocator.geocode(country)
        # And return latitude and longitude
        return (loc.latitude, loc.longitude)
    except:
        # Return missing value
        return np.nan
df["GeoLocate"]=df["codes"].apply(geolocate)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   CountryName  4 non-null      object 
 1   Age          4 non-null      int64  
 2   codes        4 non-null      object 
 3   Country      4 non-null      object 
 4   Continent    4 non-null      object 
 5   GeoLocate    4 non-null      object 
 6   Latitude     4 non-null      float64
 7   Longitude    4 non-null      float64
dtypes: float64(2), int64(1), object(5)
memory usage: 384.0+ bytes


In [34]:
def addElement(x):
    print(type(x[0]))
df["XYZ"]=df["GeoLocate"].apply(addElement)

df.head()

<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>


,CountryName,Age,codes,Country,Continent,GeoLocate,XYZ
0,United States,20,"(US, NA)",US,NA,"(43.1561681, -75.8449946)",None
1,China,21,"(CN, AS)",CN,AS,"(43.3903746, -5.7015428)",None
2,Germ,19,"(Unknown, Unknown)",Unknown,Unknown,"(29.7240421, 79.2087114)",None
3,Brazil,18,"(BR, SA)",BR,SA,"(-23.50258755, -46.62521754087413)",None


In [14]:
df["Latitude"]=df["GeoLocate"].apply(lambda x:x[0])
df["Longitude"]=df["GeoLocate"].apply(lambda x:x[1])
df.head()

,CountryName,Age,codes,Country,Continent,GeoLocate,Latitude,Longitude
0,United States,20,"(US, NA)",US,NA,"(43.1561681, -75.8449946)",43.156168,-75.844995
1,China,21,"(CN, AS)",CN,AS,"(43.3903746, -5.7015428)",43.390375,-5.701543
2,Germ,19,"(Unknown, Unknown)",Unknown,Unknown,"(29.7240421, 79.2087114)",29.724042,79.208711
3,Brazil,18,"(BR, SA)",BR,SA,"(-23.50258755, -46.62521754087413)",-23.502588,-46.625218


In [12]:
# Create a world map to show distributions of users 
import folium
from folium.plugins import MarkerCluster
#empty map
world_map= folium.Map(tiles="cartodbpositron")
marker_cluster = MarkerCluster().add_to(world_map)
#for each coordinate, create circlemarker of user percent
for i in range(len(df)):
        lat = df.iloc[i]['Latitude']
        long = df.iloc[i]['Longitude']
        radius=5
        popup_text = """Country : {}<br>
                    %of Users : {}<br>"""
        popup_text = popup_text.format(df.iloc[i]['Country'],
                                   df.iloc[i]['Age']
                                   )
        folium.CircleMarker(location = [lat, long], radius=radius, popup= popup_text, fill =True).add_to(marker_cluster)
#show the map
world_map